In [ ]:
# Copyright (c) 2025, ETH Zurich

In [1]:
import sys
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import os
import scipy.stats as stats
import scipy.ndimage
from tqdm import tqdm
import spekpy as spk
import h5py

from scipy import interpolate

In [ ]:
rave_sim_dir = Path('../rave-sim').resolve()
simulations_dir = Path('<PATH_TO_STORE_SIMULATIONS>')
scratch_dir = simulations_dir


sys.path.insert(0, str(rave_sim_dir / "nist_lookup"))
from nist_lookup.xraydb_plugin import xray_delta_beta

In [3]:
sys.path.insert(0, str(rave_sim_dir / "big-wave"))
import multisim
import config
import util
import propagation


In [4]:
def calculate_G1_height(eng):
    # constants
    h = 6.62607004 * 10**(-34) # planck constant in mˆ2 kg / s
    c_0 = 299792458 # speed of light in m / s
    eV_to_joule = 1.602176634*10**(-19)
    N_A = 6.02214086 * 10**23 #[1/mol]
    
    lambda_ = h * c_0 / (eng*eV_to_joule)
    delta_diff = xray_delta_beta('Si', 2.34, eng)[0]
    height = np.pi  * lambda_ / (2*np.pi * delta_diff)

    return height


def signal_retrieval_least_squares(data, period=None, axis=-1):
    if axis != -1:
        data = np.moveaxis(data, axis, -1)

    nsteps = data.shape[-1]

    if period is None:
        period = nsteps

    phi = np.linspace(0, 2 * np.pi * nsteps / (period), nsteps, endpoint=False)
    M = np.c_[np.sin(phi), np.cos(phi), np.ones(nsteps)]
    res, chi2, _rank, _sing_vals = np.linalg.lstsq(M, data.reshape((-1, nsteps)).T, rcond=-1)

    res = res.T.reshape((*data.shape[:-1], -1))

    dabs = res[...,2]
    dphase = -np.arctan2(res[...,0], res[...,1])
    dvis = np.sqrt(res[...,0]**2 + res[...,1]**2) / dabs

    # normalization to the total number of counts
    dabs *= nsteps

    return dabs, dphase, dvis, np.nanmean(chi2)


def calculate_pixel_intensity(x, fringe, pxEdges, statistics = 'sum'):
    fringeStats = stats.binned_statistic(x, fringe, bins=pxEdges, statistic = statistics)
    return fringeStats.statistic

def perform_binned_signal_retrieval(x, wf, pxSize, nrSteps, plot_curve = True):
    leftSide = np.arange(0-pxSize/2, np.min(x), -pxSize)
    rightSide = np.arange(0 + pxSize/2, np.max(x), pxSize)
    pxEdges = np.concatenate([np.flip(leftSide), rightSide])
    int_px = []
    for i in range(nrSteps):
        int_px.append(calculate_pixel_intensity(x, wf[i,:], pxEdges))
    int_px = np.asarray(int_px)

    trans, phase, vis, _ = signal_retrieval_least_squares(int_px, period = nrSteps, axis = 0)
    if plot_curve:
        plot_curves(trans, phase, vis, pxEdges)
    return int_px, trans, phase, vis, pxEdges

def plot_curves(trans, phase, vis, pxEdges):

    fig, axs = plt.subplots(figsize=(15,6), sharex = True, nrows = 1, ncols = 2)
    axs[0].plot(pxEdges[1:], trans, label = 'Transmission')
    axs[0].set_title('Transmission')
    axs[1].plot(pxEdges[1:], vis, label = 'Visibility')
    axs[1].set_title('Visibility')

def perform_binning(x, wf, pxSize):
    leftSide = np.arange(0-pxSize/2, np.min(x), -pxSize)
    rightSide = np.arange(0 + pxSize/2, np.max(x), pxSize)
    pxEdges = np.concatenate([np.flip(leftSide), rightSide])
    int_px = calculate_pixel_intensity(x, wf, pxEdges, statistics = 'mean')
    return int_px


    #np.random.seed(1)
    
    sample_energy = np.random.choice(x, p=y)
    if sample_energy >= low_threshold:
        return True
    else:
        return False
    
    
def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return idx

def calculate_SNR(wavefronts, detector_x, phase_steps):
    summed_wf = np.zeros_like(wavefronts[0][0])
        
    for point in wavefronts:
        wf, x_point, eng = point
        
        summed_wf += wf

    return summed_wf

def get_subdict(dict_, idx):
    sub_dict = {}
    for key in dict_.keys():
        sub_dict[key] = dict_[key][idx]
    return sub_dict

def calc_Vis_theoretical(eng, Edes,m):
    V = 2/np.pi * np.abs(np.sin(np.pi / 2 * Edes / eng)**2 * np.sin(m * np.pi / 2 * Edes / eng))
    return V

def mu_h2o(eng):
    lambda_ = h * c_0 / (eng*eV_to_joule)
    beta = xray_delta_beta('H2O', 1.0, eng)[1]
    return 4 * np.pi / lambda_ * beta


In [5]:
# constants
h = 6.62607004 * 10**(-34) # planck constant in mˆ2 kg / s
c_0 = 299792458 # speed of light in m / s
eV_to_joule = 1.602176634*10**(-19)
N_A = 6.02214086 * 10**23 #[1/mol]
E_des = 46000

lambda_ = h * c_0 / (E_des*eV_to_joule)
p2 = 4.2*10**(-6)
p0 = p1 = p2

Dn_3 = 3*p2**2/(2*lambda_) / 2
print(Dn_3)

z_g0 = 0.218
z_g1 = z_g0 + Dn_3
z_g2 = z_g0 + 2*Dn_3
z_detector = z_g2 + 0.01

h2 = 180e-6
h1 = 59e-6 

dc0 = [0.5, 0.5]
dc2 = [0.24, 0.52]

print("Z0: ", z_g0)
print("Z1: ", z_g1)
print("Z2: ", z_g2)
print("Z Detector: ", z_detector)

print(Dn_3)

0.49085288108789216
Z0:  0.218
Z1:  0.7088528810878921
Z2:  1.1997057621757843
Z Detector:  1.2097057621757843
0.49085288108789216


In [6]:
2200/ 4200

0.5238095238095238

In [7]:
N = 2**26
max_energy = 70000
dx = propagation.max_dx(z_g0, 10e-6, N, propagation.convert_energy_wavelength(max_energy))

# Now with both gratings modelled

In [ ]:
meta_data_g0 = np.load('../grating_models/DRIE_models/mask_g0_different_dc_2.npz')
meta_data_g2 = np.load('../grating_models/DRIE_models/mask_g2_different_dc_2.npz')



grid_path_g0 = os.path.abspath('../grating_models/DRIE_models/mask_g0_different_dc_2.npy')
grid_path_g2 = os.path.abspath('../grating_models/DRIE_models/mask_g2_different_dc_2.npy')


dx_g0 = meta_data_g0['x'][1].item()-meta_data_g0['x'][0].item()
dz_g0 = meta_data_g0['z'].item()

dx_g2 = meta_data_g2['x'][1].item()-meta_data_g2['x'][0].item()
dz_g2 = meta_data_g2['z'].item()

In [ ]:
s = spk.Spek(kvp=70, dk = 0.1, th = 10) # Create a spectrum

s.filter('Be', 0.15) # Create a spectrum
k, f = s.get_spectrum(edges=True) # Get the spectrum

energyRange = [4000, 70000]
dE = 100
filtering = 0.000

energies = np.arange(5, 70+0.1, 0.1)*1e3


tube_spectrum_txt = interpolate.interp1d(k*1e3, f, fill_value = 'extrapolate')
spec_txt = tube_spectrum_txt(energies)

with h5py.File('../spectra/spectrum_70_spekpy_unfiltered.h5', 'w') as h5:
    h5.create_dataset('pdf', data =  spec_txt/ np.sum(spec_txt))
    h5.create_dataset('energy', data = energies) 

path_to_spectrum = os.path.abspath('../spectra/spectrum_70_spekpy_unfiltered.h5')

In [ ]:
config_dict = {
        "sim_params": {
            "N": N,
            "dx": dx,
            "z_detector": z_g2 + 500e-6,
            "detector_size": 0.004,
            "detector_pixel_size_x": 1e-7,
            "detector_pixel_size_y": 1.0,
            "chunk_size": 256 * 1024 * 1024 // 16,  # use 256MB chunks
        },
        "use_disk_vector": False,
        "save_final_u_vectors": False,
        "dtype": "c8",
        "multisource": {
            "type": "points",
            "energy_range": [11000, 70000],
            "x_range": [-10e-6, 10e-6],
            "z": 0.0,
            "nr_source_points": 1000,
            "seed": 1,
            "spectrum": path_to_spectrum,
        },
        "elements": [
            {
                "type": "sample",
                "z_start": z_g0,
                "pixel_size_x": dx_g0,
                "pixel_size_z": dz_g0,
                "grid_path": grid_path_g0,
                "materials": [["Au", 19.32], ["Si", 2.34]],
                "x_positions": [0.0],
            },
            {
                "type": "grating",
                "pitch": p0,
                "dc": [1.0, 1.0],
                "z_start": z_g0 + 117e-6,
                "thickness": 1e-8,
                "nr_steps": 0,
                "x_positions": [0.0],
                "substrate_thickness": 370*1e-6 - 117e-6 - 1e-8,
                "mat_a": ["Si", 2.34],
                "mat_b": ["Si", 2.34],
                "mat_substrate": ["Si", 2.34],
            },
            {
                "type": "grating",
                "pitch": p1,
                "dc": [0.5, 0.5],
                "z_start": z_g1,
                "thickness": h1,
                "nr_steps": 10,
                "x_positions": [0.0],
                "substrate_thickness": 200 * 1e-6 - h1,
                "mat_a": ["Si", 2.34],
                "mat_b": None,
                "mat_substrate": ["Si", 2.34],
            },
            {
                "type": "grating",
                "pitch": p2,
                "dc": [0.5, 0.5],
                "z_start": z_g2,
                "thickness": h2,
                "nr_steps": 40,
                "x_positions": (np.arange(5) * p2/5).tolist(),
                "substrate_thickness": 500*1e-6 - h2,
                "mat_a": ["C5H8O2", 1.19],
                "mat_b": ["Au", 19.32],
                "mat_substrate": ["C", 2.26],
            },
        ],
}

sim_path = multisim.setup_simulation(config_dict, Path("."), simulations_dir)

2024-03-25 20:44:25,091 INFO: Setting up simulation
2024-03-25 20:45:23,860 INFO: Finished setting up simulation in /scratch/vieirapa/XNPIG_Data/2024/03/20240325_204521593776


In [ ]:
for i in range(1000):
    os.system(f"CUDA_VISIBLE_DEVICES=1 ../rave-sim/fast-wave/build-Release/fastwave -s {i} {sim_path}")

[2024-03-25 20:45:24.249] [info] Running simulation /scratch/vieirapa/XNPIG_Data/2024/03/20240325_204521593776/00000000
[2024-03-25 20:45:24.684] [info] Simulating optical element 1/4
[2024-03-25 20:45:30.072] [info] Elapsed time for optical element: 5388.276 ms
[2024-03-25 20:45:30.072] [info] Simulating optical element 2/4
[2024-03-25 20:45:30.095] [info] Elapsed time for optical element: 22.8864 ms
[2024-03-25 20:45:30.116] [info] Simulating optical element 3/4
[2024-03-25 20:45:30.389] [info] Elapsed time for optical element: 273.0407 ms
[2024-03-25 20:45:30.413] [info] Simulating optical element 4/4
[2024-03-25 20:45:31.435] [info] Elapsed time for optical element: 1022.72797 ms
[2024-03-25 20:45:31.437] [info] Running phase step 2/5
[2024-03-25 20:45:31.439] [info] Simulating optical element 4/4
[2024-03-25 20:45:32.463] [info] Elapsed time for optical element: 1024.6134 ms
[2024-03-25 20:45:32.464] [info] Running phase step 3/5
[2024-03-25 20:45:32.466] [info] Simulating optical

In [ ]:
h0 = 180e-6

In [ ]:
config_dict = {
        "sim_params": {
            "N": N,
            "dx": dx,
            "z_detector": z_g2 + 370e-6,
            "detector_size": 0.004,
            "detector_pixel_size_x": 1e-7,
            "detector_pixel_size_y": 1.0,
            "chunk_size": 256 * 1024 * 1024 // 16,  # use 256MB chunks
        },
        "use_disk_vector": False,
        "save_final_u_vectors": False,
        "dtype": "c8",
        "multisource": {
            "type": "points",
            "energy_range": [11000, 70000],
            "x_range": [-10e-6, 10e-6],
            "z": 0.0,
            "nr_source_points": 1000,
            "seed": 1,
            "spectrum": path_to_spectrum,
        },
        "elements": [
            {
                "type": "grating",
                "pitch": p0,
                "dc": [0.5, 0.5],
                "z_start": z_g0,
                "thickness": h0,
                "nr_steps": 30,
                "x_positions": [0.0],
                "substrate_thickness": 500*1e-6 - h0,
                "mat_a": ["C5H8O2", 1.19],
                "mat_b": ["Au", 19.32],
                "mat_substrate": ["C", 2.26],
            },
            {
                "type": "grating",
                "pitch": p1,
                "dc": [0.5, 0.5],
                "z_start": z_g1,
                "thickness": h1,
                "nr_steps": 10,
                "x_positions": [0.0],
                "substrate_thickness": 200 * 1e-6 - h1,
                "mat_a": ["Si", 2.34],
                "mat_b": None,
                "mat_substrate": ["Si", 2.34],
            },
            {
                "type": "sample",
                "z_start": z_g2,
                "pixel_size_x": dx_g2,
                "pixel_size_z": dz_g2,
                "grid_path": grid_path_g2,
                "materials": [["Au", 19.32], ["Si", 2.34]],
                "x_positions": (np.arange(5) * p2/5).tolist(),
            },
            {
                "type": "grating",
                "pitch": p2,
                "dc": [1.0, 1.0],
                "z_start": z_g2 + 147e-6,
                "thickness": 1e-10,
                "nr_steps": 0,
                "x_positions": (np.arange(5) * p2/5).tolist(),
                "substrate_thickness": 370*1e-6 - 147e-6,
                "mat_a": ["Si", 2.34],
                "mat_b": ["Si", 2.34],
                "mat_substrate": ["Si", 2.34],
            },
        ],
}

sim_path = multisim.setup_simulation(config_dict, Path("."), simulations_dir)

In [ ]:
for i in range(1000):
    os.system(f"CUDA_VISIBLE_DEVICES=1 ../rave-sim/fast-wave/build-Release/fastwave -s {i} {sim_path}")